In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from IPython.display import Markdown, display
import gradio as gr

In [2]:
load_dotenv(override=True)

True

In [49]:
import requests
from bs4 import BeautifulSoup

def get_top_news(url="https://www.tagesschau.de/"):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # Find the main news section
    articles = soup.select("a.teaser__link")[:3]  # Select top 3 teaser links

    news = []
    for article in articles:
        title = article.get_text(strip=True)
        news.append({"title": title})

    return news


In [50]:
top_news = get_top_news()
for i, item in enumerate(top_news, 1):
    print(f"{i}. {item['title']}")

1. faqBundesverwaltungsgerichtWorum es bei der Entscheidung über Compact gehtDas Bundesinnenministerium will das rechtsextreme Compact-Magazin verbieten. Doch ein erster Anlauf scheiterte 2024. Nun soll das Bundesverwaltungsgericht entscheiden. Ein Urteil könnte noch in dieser Woche fallen.mehr
2. Erste GruppeUkraine und Russland tauschenKriegsgefangeneausNach tagelanger Verzögerung haben die Ukraine und Russland eine erste Gruppe Kriegsgefangener ausgetauscht. Doch auch die schweren Angriffe gehen weiter: Die Ukraine meldete einen massiven nächtlichen  Drohnenangriff Russlands.mehr
3. ReportageGeflohen aus MariupolZwischen Heimweh und HoffnungBis zum russischen Angriff auf die Ukraine betrieb Wjatscheslaw Dolschenko in Mariupol ein Museum. Doch nun ist alles zerstört. Er selbst musste fliehen - und hofft darauf, eines Tages zurückkehren zu können.Von A. Beer und E. Samowilowa.mehr


## Tool 1

In [3]:
instructions_news_finder = "summarize 3 news that was reported in 2024 Germany in German Language. \
    The news can be old. \
    The language must be German,\
    Summurize each in 3 short sentences.\
    return the date of news\
    only return 3 markdown item. No explanation in the begining or the end."

In [4]:
agent_news_finder = Agent(name="News Finder Agent" , instructions=instructions_news_finder, model="gpt-4o-mini")

In [5]:
tool_news_finder = agent_news_finder.as_tool(tool_name="News_finder_tool", tool_description="find 5 german news in 2024.")

## Tool 2

In [6]:
instructions_word_finder = """You are a German Language instructor who reads 3 short news in German and finds 4 most difficult German words.\
    you are given 3 short news in German.
    as output, return 4 most advanced words of each news in German without English translation.\
    return 3 markdown items for each news and list the 4 words that you find below each item. 
    The output must contain 12 German words.
    No explanation in the begining or the end.
    """

In [7]:
agent_word_finder = Agent(name="Word Finder Agent", 
                          instructions=instructions_word_finder, 
                          model="gpt-4o-mini",
                          tools=[tool_news_finder])

In [24]:
tool_word_finder = agent_word_finder.as_tool(tool_name="word_finder_tool", tool_description="find 5 advanced word of each news")

# Tool 3

In [29]:
instructions_german_to_english = """
You are a German Language instructor who translates 4 most difficult German words to English for each news.\
    you are given 3 short news in German and 4 most difficult German words for each news.\
    as output, return 3 markdown items for each news. \
    each item must contain the short news, date, and 4 Gertman words with their article (der, die, das) and einglish translation and explanation of the word in the context of the news.\
    return the article in parnthesis after the word and before the translation
    only return the article for the nouns not for the verbs and not adjectives\
    No explanation in the begining or the end.
    """

In [30]:
agent_german_to_english = Agent(name="German to English Translator", instructions=instructions_german_to_english, model="gpt-4o-mini")

In [31]:
tool_german_to_english = agent_german_to_english.as_tool(tool_name="german_to_english_tool", 
                                                         tool_description="translate 5 most difficult German words for each news")

## Final Agent

In [32]:
instructions_german_instructor = """You are a German Language instructor who reads 3 short news in German and translates 4 most difficult German words for each news.\
    The 3 short news are given to you using the "News_finder_tool" tool\
    the 4 most difficult words are given to you using the "word_finder_tool" tool\
    The meaning and explanation of the word is given to you using the "german_to_english_tool" tool\
    only use the tools to access input news and words\
    as output, return 3 markdown items for each news. \
    each item must contain the short news, date, and 4 Gertman words with their article (der, die, das) and einglish translation and explanation of the word in the context of the news.\
    return the article in parnthesis after the word and before the translation
    only return the article for the nouns not for the verbs and not adjectives\
    No explanation in the begining or the end.
    """

In [33]:
agent_german_instructor = Agent(name="German Instructor Agent", 
                                instructions=instructions_german_instructor, 
                                model="gpt-4o-mini",
                                tools=[tool_news_finder, tool_word_finder])

In [34]:
with trace("German Instructor"):
    result = await Runner.run(agent_german_instructor, "Translate 5 most difficult words")
    display(Markdown(result.final_output)) 

### 1. News vom 15. Januar 2024
Die deutsche Regierung hat ein neues Gesetz zur Förderung der erneuerbaren Energien verabschiedet. Ziel ist es, die Abhängigkeit von fossilen Brennstoffen bis 2030 um 50% zu reduzieren. Kritiker warnen jedoch vor hohen Kosten für die Verbraucher.
- **Abhängigkeit** (die) - dependency; refers to the reliance on certain energy sources.
- **fossile** (die) - fossil; describes fuels that are derived from the earth, highlighting their environmental impact.
- **Brennstoffe** (die) - fuels; refers specifically to energy sources used in generation.
- **Verbraucher** (der) - consumer; relates to individuals who utilize energy and are affected by costs.

### 2. News vom 22. Februar 2024
Die Bundesregierung kündigt ein neues Programm zur Bekämpfung der Wohnungsnot an. Es sollen günstige Mietwohnungen in städtischen Gebieten gebaut und subventioniert werden. Dies wird als wichtiger Schritt zur Verbesserung der Lebensqualität in Städten gesehen.
- **Wohnungsnot** (die) - housing shortage; indicates the lack of affordable living spaces.
- **Mietwohnungen** (die) - rental apartments; refers to available living spaces for rent.
- **subventioniert** (die) - subsidized; describes financial assistance to promote affordable housing.
- **Lebensqualität** (die) - quality of life; highlights the improvement in living conditions.

### 3. News vom 10. April 2024
Die Debatte um die Digitalisierung in Schulen nimmt an Fahrt auf. Experten fordern schnellere Investitionen in digitale Infrastruktur und Schulungen für Lehrkräfte. Das Ziel ist, den Schülern moderne Kompetenzen für die Zukunft zu vermitteln.
- **Digitalisierung** (die) - digitalization; relates to integrating digital technologies in education.
- **Infrastruktur** (die) - infrastructure; refers to the foundational systems needed for effective digital implementation.
- **Schulungen** (die) - training; describes the education and preparation for teachers in digital methods.
- **Kompetenzen** (die) - competencies; designates the skills students should develop in a modern learning environment.

In [18]:
def run():
    return result.final_output 

In [19]:
with gr.Blocks(theme=gr.themes.Default(primary_hue="sky")) as ui:
    gr.Markdown("# Learn German with News!")
    # query_textbox = gr.Textbox(label="What topic would you like to research?")
    run_button = gr.Button("Display 3 short news with 4 most difficult words!", variant="primary")
    report = gr.Markdown(label="Report")
    run_button.click(fn=run, outputs=report)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7860/: Operation not supported
